In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats  # For plotting theoretical distributions

In [ ]:
# Linear Congruential Generator (LCG) Class
class LCG:
    def __init__(self, seed, a=1664525, c=1013904223, m=2**32):
        self.seed = seed
        self.a = a
        self.c = c
        self.m = m
        self.current = seed

    def random(self):
        """
        Generates a pseudo-random number between 0 and 1.
        Every time you call lcg.random(), the current value is updated according to the linear congruential formula: 
        X_{n+1} = (a * X_n + c) mod m
        """
        self.current = (self.a * self.current + self.c) % self.m
        return self.current / self.m  # Normalize to [0, 1)


In [ ]:

# Box-Muller Transform for Normal Distribution
def box_muller(lcg):
    u1 = lcg.random()
    u2 = lcg.random()
    z1 = math.sqrt(-2.0 * math.log(u1)) * math.cos(2.0 * math.pi * u2)
    z2 = math.sqrt(-2.0 * math.log(u1)) * math.sin(2.0 * math.pi * u2)
    return z1, z2

#### Continuous Distributions
def generate_normal_demand(lcg, mu=0, sigma=1):
    z1, _ = box_muller(lcg)
    return mu + sigma * z1

def generate_uniform_demand(lcg, a=0, b=1):
    u = lcg.random()
    return a + (b - a) * u

def generate_exponential_demand(lcg, lam=1):
    u = lcg.random()
    return -math.log(u) / lam

def generate_lognormal_demand(lcg, mu=0, sigma=1):
    z1, _ = box_muller(lcg)
    return math.exp(mu + sigma * z1)

def generate_triangular_demand(lcg, a=0, c=0.5, b=1):
    u = lcg.random()
    if u < (c - a) / (b - a):
        return a + math.sqrt(u * (b - a) * (c - a))
    else:
        return b - math.sqrt((1 - u) * (b - a) * (b - c))

def generate_beta_demand(lcg, alpha=2, beta=5):
    x = np.random.gamma(alpha, 1)
    y = np.random.gamma(beta, 1)
    return x / (x + y)

def generate_weibull_demand(lcg, lam=1, k=1.5):
    u = lcg.random()
    return lam * (-math.log(1 - u))**(1 / k)

#### Discrete Distributions
def generate_poisson_demand(lcg, lam=1):
    L = math.exp(-lam)
    k = 0
    p = 1.0
    while p > L:
        k += 1
        u = lcg.random()
        p *= u
    return k - 1

def generate_binomial_demand(lcg, n=10, p=0.5):
    successes = 0
    for _ in range(n):
        if lcg.random() < p:
            successes += 1
    return successes

def generate_bernoulli_demand(lcg, p=0.5):
    return 1 if lcg.random() < p else 0

def generate_geometric_demand(lcg, p=0.5):
    u = lcg.random()
    return math.ceil(math.log(1 - u) / math.log(1 - p))

def generate_negative_binomial_demand(lcg, r=10, p=0.5):
    successes = 0
    trials = 0
    while successes < r:
        if lcg.random() < p:
            successes += 1
        trials += 1
    return trials


In [ ]:
# Function to simulate demand for any distribution
def simulate_demand(lcg, iterations, distribution='normal', **kwargs):
    demands = []
    for _ in range(iterations):
        if distribution == 'normal':
            demand = generate_normal_demand(lcg, kwargs.get('mu', 0), kwargs.get('sigma', 1))
        elif distribution == 'uniform':
            demand = generate_uniform_demand(lcg, kwargs.get('a', 0), kwargs.get('b', 1))
        elif distribution == 'exponential':
            demand = generate_exponential_demand(lcg, kwargs.get('lam', 1))
        elif distribution == 'poisson':
            demand = generate_poisson_demand(lcg, kwargs.get('lam', 1))
        elif distribution == 'binomial':
            demand = generate_binomial_demand(lcg, kwargs.get('n', 10), kwargs.get('p', 0.5))
        elif distribution == 'lognormal':
            demand = generate_lognormal_demand(lcg, kwargs.get('mu', 0), kwargs.get('sigma', 1))
        elif distribution == 'bernoulli':
            demand = generate_bernoulli_demand(lcg, kwargs.get('p', 0.5))
        elif distribution == 'geometric':
            demand = generate_geometric_demand(lcg, kwargs.get('p', 0.5))
        elif distribution == 'negative_binomial':
            demand = generate_negative_binomial_demand(lcg, kwargs.get('r', 10), kwargs.get('p', 0.5))
        elif distribution == 'triangular':
            demand = generate_triangular_demand(lcg, kwargs.get('a', 0), kwargs.get('c', 0.5), kwargs.get('b', 1))
        elif distribution == 'beta':
            demand = generate_beta_demand(lcg, kwargs.get('alpha', 2), kwargs.get('beta', 5))
        elif distribution == 'weibull':
            demand = generate_weibull_demand(lcg, kwargs.get('lam', 1), kwargs.get('k', 1.5))
        else:
            raise ValueError(f"Unknown distribution: {distribution}")
        demands.append(demand)
    return np.array(demands)

# Plot demand and theoretical distribution
def plot_demand_distribution(demands, distribution, **kwargs):
    plt.hist(demands, bins=50, density=True, alpha=0.6, color='g', label='Simulated Demand')
    x = np.linspace(min(demands), max(demands), 1000)

    if distribution == 'normal':
        plt.plot(x, stats.norm.pdf(x, kwargs.get('mu', 0), kwargs.get('sigma', 1)), 'r-', lw=2, label='Normal PDF')
    elif distribution == 'uniform':
        a, b = kwargs.get('a', 0), kwargs.get('b', 1)
        plt.plot(x, stats.uniform.pdf(x, a, b - a), 'r-', lw=2, label='Uniform PDF')
    elif distribution == 'exponential':
        lam = kwargs.get('lam', 1)
        plt.plot(x, stats.expon.pdf(x, scale=1/lam), 'r-', lw=2, label='Exponential PDF')
    elif distribution == 'poisson':
        lam = kwargs.get('lam', 1)
        k = np.arange(0, max(demands) + 1)
        plt.bar(k, stats.poisson.pmf(k, lam), alpha=0.75, label='Poisson PMF')
    elif distribution == 'lognormal':
        mu, sigma = kwargs.get('mu', 0), kwargs.get('sigma', 1)
        plt.plot(x, stats.lognorm.pdf(x, sigma, scale=np.exp(mu)), 'r-', lw=2, label='Lognormal PDF')
    elif distribution == 'triangular':
        a, c, b = kwargs.get('a', 0), kwargs.get('c', 0.5), kwargs.get('b', 1)
        plt.plot(x, stats.triang.pdf(x, (c - a) / (b - a), loc=a, scale=b - a), 'r-', lw=2, label='Triangular PDF')
    elif distribution == 'beta':
        alpha, beta = kwargs.get('alpha', 2), kwargs.get('beta', 5)
        plt.plot(x, stats.beta.pdf(x, alpha, beta), 'r-', lw=2, label='Beta PDF')
    elif distribution == 'weibull':
        lam, k = kwargs.get('lam', 1), kwargs.get('k', 1.5)
        plt.plot(x, stats.weibull_min.pdf(x, k, scale=lam), 'r-', lw=2, label='Weibull PDF')
    elif distribution == 'geometric':
        p = kwargs.get('p', 0.5)
        k = np.arange(1, max(demands) + 1)
        plt.bar(k, stats.geom.pmf(k, p), alpha=0.75, label='Geometric PMF')
    elif distribution == 'negative_binomial':
        r, p = kwargs.get('r', 10), kwargs.get('p', 0.5)
        k = np.arange(0, max(demands) + 1)
        plt.bar(k, stats.nbinom.pmf(k, r, p), alpha=0.75, label='Negative Binomial PMF')
    elif distribution == 'bernoulli':
        p = kwargs.get('p', 0.5)
        k = [0, 1]
        plt.bar(k, stats.bernoulli.pmf(k, p), alpha=0.75, label='Bernoulli PMF')
 
    plt.title(f"Demand Distribution: {distribution.capitalize()}")
    plt.xlabel('Demand')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True)
    plt.show()

# Newsvendor simulation with any distribution
def simulate_newsvendor(Q, iterations, distribution='normal', **kwargs):
    lcg = LCG(seed=1234)
    profits = []
    for _ in range(iterations):
        demand = simulate_demand(lcg, 1, distribution, **kwargs)[0]
        if demand <= Q:
            profit = p * demand - c * Q + v * (Q - demand)
        else:
            profit = p * Q - c * Q
        profits.append(profit)
    return np.mean(profits)


In [ ]:

# Define general parameters for the simulation
iterations = 10000  # Number of iterations for the Monte Carlo simulation
distribution = 'negative_binomial'  # Specify the distribution to use

# Define specific parameters for each distribution
distribution_params = {
    'normal': {'mu': 100, 'sigma': 20},
    'uniform': {'a': 50, 'b': 150},
    'exponential': {'lam': 1/100},
    'poisson': {'lam': 100},
    'lognormal': {'mu': 4.5, 'sigma': 0.75},
    'triangular': {'a': 50, 'c': 100, 'b': 150},
    'beta': {'alpha': 2, 'beta': 5},
    'weibull': {'lam': 1.5, 'k': 1.2},
    'geometric': {'p': 0.3},
    'negative_binomial': {'r': 10, 'p': 0.4},
    'bernoulli': {'p': 0.5},
    'binomial': {'n': 10, 'p': 0.5}
}

# Initialize the LCG with a seed
lcg = LCG(seed=1234)

# Simulate demand based on the specified distribution and parameters
demands = simulate_demand(lcg, iterations, distribution, **distribution_params[distribution])

# Plot the simulated demand along with the theoretical distribution
plot_demand_distribution(demands, distribution, **distribution_params[distribution])

# Simulate the newsvendor problem for different order quantities
order_quantities = np.arange(80, 121)
average_profits = [simulate_newsvendor(Q, iterations, distribution, **distribution_params[distribution]) for Q in order_quantities]

# Plot the newsvendor simulation results
plt.plot(order_quantities, average_profits)
plt.xlabel('Order Quantity (Q)')
plt.ylabel('Average Profit')
plt.title(f'Newsvendor Problem: Monte Carlo Simulation ({distribution.capitalize()} Distribution)')
plt.grid(True)
plt.show()
